In [1]:
import pandas as pd
import networkx as nx
from node2vec import Node2Vec
from sklearn.cluster import DBSCAN
from tqdm import tqdm

In [2]:
addres_df = pd.read_csv('all_locations.csv')

In [3]:
df = pd.read_csv('EVL_pre-network_ZA-BOSKO.csv')

In [4]:
df2021 = df.groupby(['LOKACIJA_ODDAJE','LOKACIJA_PREJEMA']).size().reset_index()#.reset_index()#.size()

In [5]:
df2021

,LOKACIJA_ODDAJE,LOKACIJA_PREJEMA,0
0,"ABRAMOVA ULICA 26, 1000 LJUBLJANA","AGROKOMBINATSKA CESTA 80, 1000 LJUBLJANA",84
1,"ABRAMOVA ULICA 26, 1000 LJUBLJANA",Brez naslova,60
2,"ABRAMOVA ULICA 26, 1000 LJUBLJANA","CESTA DVEH CESARJEV 101, 1000 LJUBLJANA",3
3,"ABRAMOVA ULICA 26, 1000 LJUBLJANA","CESTA V PROD 84, 1000 LJUBLJANA",1
4,"ABRAMOVA ULICA 26, 1000 LJUBLJANA","KAMNJE 41, 8232 ŠENTRUPERT",2
...,...,...,...
76814,"ŽVAB 16, 2259 IVANJKOVCI","SPODNJI PORČIČ 4A, 2230 LENART V SLOV. GORICAH",23
76815,"ŽVIRČE 10A, 8362 HINJE","LJUBLJANSKA CESTA 45, 1241 KAMNIK",1
76816,"ŽVIRČE 10A, 8362 HINJE","NOVOMEŠKA CESTA 7, 1330 KOČEVJE",1
76817,"ŽVIRČE 22, 8362 HINJE","GLINJE 17, 4207 CERKLJE NA GORENJSKEM",5


In [6]:
from math import sin, cos, sqrt, atan2

def get_dist(lat1,lon1,lat2,lon2):
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

In [7]:
G = nx.DiGraph()

In [8]:
import math

In [9]:
for i,row in tqdm(df2021.iterrows(), total=len(df2021)):
    x = row['LOKACIJA_ODDAJE']
    y = row['LOKACIJA_PREJEMA']
   # if i == 50:
   #     break
    try:
        x_lang = addres_df[addres_df['Lokacija'] == x].lang.to_list()[0]
        x_long  = addres_df[addres_df['Lokacija'] == x].long.to_list()[0]
        y_lang = addres_df[addres_df['Lokacija'] ==y].lang.to_list()[0]
        y_long = addres_df[addres_df['Lokacija'] == y].long.to_list()[0]
        dist = get_dist(x_lang,x_long,y_lang,y_long)
        if math.isnan(dist):
            #print(dist)
            continue
        G.add_node(x, lang = x_lang, long =  x_long)
        G.add_node(y, lang = y_lang, long = y_long)
        G.add_edge(x, y,weight = dist)
    except Exception as e :
        #print(e)
        pass

100%|███████████████████████████████████████████████████████████████████████████| 76819/76819 [07:18<00:00, 175.06it/s]


In [ ]:
with open('graph.pkl','wb') as f:
    import pickle as p
    p.dump(G,f)

In [10]:
A = nx.adjacency_matrix(G)

In [ ]:
!pip install hdbscan

In [11]:
import hdbscan
#from sklearn.datasets import make_blobs

clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
labels = clusterer.fit_predict(A)

In [54]:
len(set(labels))

490

In [69]:
for e in [5000,6700,8000,10000]:
    for z in [1]:#,2,3,4,5,10]:
        clus = DBSCAN(eps=e, min_samples=z).fit(A)
        from collections import Counter
        print("R: ",e," min_samples: ",z,Counter(clus.labels_)[-1],len(set(clus.labels_)))
        labels = clus.labels_
        label_map = {i:[] for i in set(labels)}
        for node, label in zip(G.nodes.data(), labels):
           # print(node)
            node_name = node[0]
            node_stuff = node[1]
            label_map[label].append(((node_stuff['lang'],node_stuff['long'])))
        name_map = {}
        for i, node in enumerate(G.nodes.data()):
            name_map[node[0]] = labels[i]
        import numpy as np
        final_mapper = {}
        for color in label_map:
            arr = np.array(label_map[color])
            xs,ys = np.median(arr[:,0]), np.median(arr[:,1])
            final_mapper[color] = (xs,ys)    
        df['DB_CLUSTER_LOCATION_ODDAJA'] = df['LOKACIJA_ODDAJE'].apply(lambda x: name_map.get(x,100001))#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA
        df['DB_CLUSTER_LOCATION_ODDAJA_X'] = df['DB_CLUSTER_LOCATION_ODDAJA'].apply(lambda x: final_mapper.get(x,[100001,100001])[0])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA
        df['DB_CLUSTER_LOCATION_ODDAJA_Y'] = df['DB_CLUSTER_LOCATION_ODDAJA'].apply(lambda x: final_mapper.get(x,[100001,100001])[1])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA
        df['DB_CLUSTER_LOCATION_PREJEMA'] = df['LOKACIJA_PREJEMA'].apply(lambda x: name_map.get(x,100001))#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA
        df['DB_CLUSTER_LOCATION_PREJEMA_X'] = df['DB_CLUSTER_LOCATION_PREJEMA'].apply(lambda x: final_mapper.get(x,[100001,100001])[0])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA
        df['DB_CLUSTER_LOCATION_PREJEMA_Y'] = df['DB_CLUSTER_LOCATION_PREJEMA'].apply(lambda x: final_mapper.get(x,[100001,100001])[1])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA
        df.groupby(['DAT_ODDAJA', 'DAT_PREJEM_ZAV','DB_CLUSTER_LOCATION_ODDAJA','DB_CLUSTER_LOCATION_PREJEMA','IZVOR ODPADKA','NAZIV_ODPADKA_MERGED','NEVAREN','DB_CLUSTER_LOCATION_ODDAJA_X','DB_CLUSTER_LOCATION_ODDAJA_Y','DB_CLUSTER_LOCATION_PREJEMA_X','DB_CLUSTER_LOCATION_PREJEMA_Y']).agg({'KOLIČINA ODPADKA v kg':"sum"}).reset_index().to_csv(f'EVL_boshko_done_DB-{e}.csv')
        #break

R:  5000  min_samples:  1 0 392
R:  6700  min_samples:  1 0 227
R:  8000  min_samples:  1 0 146
R:  10000  min_samples:  1 0 76


In [15]:
label_map = {i:[] for i in set(labels)}
for node, label in zip(G.nodes.data(), labels):
   # print(node)
    node_name = node[0]
    node_stuff = node[1]
    label_map[label].append(((node_stuff['lang'],node_stuff['long'])))
        

In [16]:
name_map = {}
for i, node in enumerate(G.nodes.data()):
    name_map[node[0]] = labels[i]
    

In [17]:
import numpy as np
final_mapper = {}
for color in label_map:
    arr = np.array(label_map[color])
    xs,ys = np.median(arr[:,0]), np.median(arr[:,1])
    final_mapper[color] = (xs,ys)

In [19]:
len(final_mapper.values()), len(set(final_mapper.values()))

(490, 490)

In [50]:
df['DB_CLUSTER_LOCATION_ODDAJA'].value_counts()

-1      534346
 14       6404
 24       5045
 226      2731
 150      2591
         ...  
 308        10
 454        10
 328        10
 247         7
 69          7
Name: DB_CLUSTER_LOCATION_ODDAJA, Length: 490, dtype: int64

# СТВАРНО ЈАКА Е!

CLUSTER_LOCATION_ODDAJA_X

In [ ]:
'CLUSTER_LOCATION_ODDAJA_X','CLUSTER_LOCATION_ODDAJA_Y','CLUSTER_LOCATION_PREJEMA_X','CLUSTER_LOCATION_PREJEMA_Y'

In [38]:
df['DB_CLUSTER_LOCATION_ODDAJA'] = df['LOKACIJA_ODDAJE'].apply(lambda x: name_map.get(x,100001))#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA

In [39]:
df['DB_CLUSTER_LOCATION_ODDAJA_X'] = df['DB_CLUSTER_LOCATION_ODDAJA'].apply(lambda x: final_mapper.get(x,[100001,100001])[0])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA

In [40]:
df['DB_CLUSTER_LOCATION_ODDAJA_Y'] = df['DB_CLUSTER_LOCATION_ODDAJA'].apply(lambda x: final_mapper.get(x,[100001,100001])[1])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA

In [41]:
df['DB_CLUSTER_LOCATION_PREJEMA'] = df['LOKACIJA_PREJEMA'].apply(lambda x: name_map.get(x,100001))#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA

In [42]:
df['DB_CLUSTER_LOCATION_PREJEMA_X'] = df['DB_CLUSTER_LOCATION_PREJEMA'].apply(lambda x: final_mapper.get(x,[100001,100001])[0])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA

In [43]:
df['DB_CLUSTER_LOCATION_PREJEMA_Y'] = df['DB_CLUSTER_LOCATION_PREJEMA'].apply(lambda x: final_mapper.get(x,[100001,100001])[1])#[name_map[x] for x in df['LOKACIJA_ODDAJE']]LOKACIJA_PREJEMA

In [47]:
len(set(df['DB_CLUSTER_LOCATION_PREJEMA']).union(df['DB_CLUSTER_LOCATION_ODDAJA']))

490

In [45]:
df = df[(df['DB_CLUSTER_LOCATION_ODDAJA'] != 100001) & (df['DB_CLUSTER_LOCATION_PREJEMA'] != 100001) ].reindex()

In [46]:
df.keys()

Index(['LOKACIJA_ODDAJE', 'LOKACIJA_PREJEMA', 'DAT_ODDAJA', 'DAT_PREJEM_ZAV',
       'NAZIV_ODPADKA_MERGED', 'KOLIČINA ODPADKA v kg', 'IZVOR ODPADKA',
       'NEVAREN', 'DB_CLUSTER_LOCATION_ODDAJA', 'DB_CLUSTER_LOCATION_ODDAJA_X',
       'CLUSTER_LOCATION_ODDAJA_Y', 'DB_CLUSTER_LOCATION_PREJEMA',
       'DB_CLUSTER_LOCATION_PREJEMA_X', 'DB_CLUSTER_LOCATION_PREJEMA_Y',
       'DB_CLUSTER_LOCATION_ODDAJA_Y'],
      dtype='object')

# DECISION TREES AND SVM >>> BIOBERT SCIENCE!

In [49]:
df.groupby(['DAT_ODDAJA', 'DAT_PREJEM_ZAV','DB_CLUSTER_LOCATION_ODDAJA','DB_CLUSTER_LOCATION_PREJEMA','IZVOR ODPADKA','NAZIV_ODPADKA_MERGED','NEVAREN','DB_CLUSTER_LOCATION_ODDAJA_X','DB_CLUSTER_LOCATION_ODDAJA_Y','DB_CLUSTER_LOCATION_PREJEMA_X','DB_CLUSTER_LOCATION_PREJEMA_Y']).agg({'KOLIČINA ODPADKA v kg':"sum"}).reset_index().to_csv('EVL_boshko_done_DB-fixed.csv')

In [ ]:
df_mapper2 = pd.DataFrame()

In [ ]:
final_mapper

In [ ]:
list(G.nodes)

In [226]:
node2vec = Node2Vec(G, dimensions=2, walk_length=20, num_walks=10,workers=1)
model = node2vec.fit(window=5, min_count=1)
model.wv.save_word2vec_format("embedding.emb")


Computing transition probabilities:  32%|██▌     | 4005/12383 [00:00<00:01, 4193.60it/s]


KeyboardInterrupt: 

In [192]:
from sklearn.cluster import KMeans
import numpy as np


kmeans = KMeans(n_clusters=200, random_state=0).fit(A) # apply kmeans on Z
labels=kmeans.labels_  # get the cluster labels of the nodes.
print(len(set(labels)))

200


In [247]:
max(labels)

199

In [ ]:

X = np.loadtxt("embedding.emb", skiprows=1) # load the embedding of the nodes of the graph
#print(X)
# sort the embedding based on node index in the first column in X
X=X[X[:,0].argsort()]; 
#print(X)
Z=X[0:X.shape[0],1:X.shape[1]]; # remove the node index from X and save in Z

kmeans = KMeans(n_clusters=200, random_state=0).fit(A) # apply kmeans on Z
labels=kmeans.labels_  # get the cluster labels of the nodes.
print(labels)